## Zip to S3

This notebook automates the (re)build, install, zip, upload to S3, and aws cli command to publish new version to a AWS layer.

In [7]:
from my_utils.os_utils import subprocess_execute
import shutil

In [10]:
# list of packages to build
packages_todo = [
    # list of tuples: ('package_name','from_pip')
    #('pandas',True),
    ('my_utils',False),
    ('my_dev',False),    
]
package_base_dir = '/home/jovyan/packages'
work_dir = '/home/jovyan/work'
s3_path = 's3://andrewm4894-python-packages/latest/'
s3_region = 'us-west-2'
s3_bucket = s3_path.split('s3://')[1].split('/')[0]
s3_url = s3_path.replace('s3://','https://').replace(s3_bucket,f'{s3_bucket}.s3-{s3_region}.amazonaws.com')
s3_key = 'latest/python.zip'
aws_layer_name = 'my-python-packages'

## Build & Install Packages

In [14]:
# build and install locally each package
for package_name, from_pip in packages_todo:
    cmd = ''
    # run pip install within the python dir
    if from_pip:
        # if from pip then just use name
        cmd = cmd + f'pip install --upgrade --force-reinstall "{package_name}" -t {work_dir}/python/.'
    else:
        # cd into the package dir
        cmd = f'cd {package_base_dir}/{package_name}'
        # run the build commands to create local egg file to be zipped up into an AWS layer
        cmd = cmd + f' | python setup.py bdist_egg'
        # otherwise point to local dir
        #cmd = cmd + f' | pip install --upgrade "{package_base_dir}/{package_name}" -t {work_dir}/python/.'
    # kick off big command to do the stuff
    result = subprocess_execute(cmd)
    # look at results
    print(result)

... cmd to execute:
cd /home/jovyan/packages/my_utils/ | python setup.py bdist_egg
... result:
Error: failed to execute command: cd /home/jovyan/packages/my_utils/ | python setup.py bdist_egg
b"/bin/sh: 1: cd: can't cd to /home/jovyan/packages/my_utils/\npython: can't open file 'setup.py': [Errno 2] No such file or directory\n"

... cmd to execute:
cd /home/jovyan/packages/my_dev/ | python setup.py bdist_egg
... result:
Error: failed to execute command: cd /home/jovyan/packages/my_dev/ | python setup.py bdist_egg
b"/bin/sh: 1: cd: can't cd to /home/jovyan/packages/my_dev/\npython: can't open file 'setup.py': [Errno 2] No such file or directory\n"



## Create python.zip

In [4]:
# make zip
shutil.make_archive(f'{work_dir}/python', 'zip', '/home/jovyan/python/')

'/home/jovyan/work/python.zip'

## Upload to S3

In [5]:
# upload zip to s3 using aws cli
cmd = f'aws s3 cp --region {s3_region} python.zip {s3_path}'
print(subprocess_execute(cmd))
print(f'... zip should be updated at: {s3_url}python.zip ...')

... cmd to execute:
aws s3 cp --region us-west-2 python.zip s3://andrewm4894-python-packages/latest/
... result:
upload: ./python.zip to s3://andrewm4894-python-packages/latest/python.zip

... zip should be updated at: https://andrewm4894-python-packages.s3-us-west-2.amazonaws.com/latest/python.zip ...


## Publish to AWS Layer

In [6]:
# publish zip file from s3 location to aws layer
cmd = f'aws lambda publish-layer-version --layer-name {aws_layer_name} --content S3Bucket={s3_bucket},S3Key={s3_key} --region {s3_region}'
subprocess_execute(cmd)

... cmd to execute:
aws lambda publish-layer-version --layer-name my-python-packages --content S3Bucket=andrewm4894-python-packages,S3Key=latest/python.zip --region us-west-2
... result:


'{\n    "Content": {\n        "Location": "https://awslambda-us-west-2-layers.s3.us-west-2.amazonaws.com/snapshots/908566381001/my-python-packages-2bf812ec-b31f-4f6f-b74b-86740462bdcb?versionId=x4WOQrphgucwCnVkGaKR3ZyJcvxjd19H&X-Amz-Security-Token=AgoJb3JpZ2luX2VjEEsaCXVzLXdlc3QtMiJHMEUCIQDQEvI2PhdKreOqF%2FFyC2o6J3eWavNTrCuyWmJDBaESpwIgVVF7%2B1MHL4NHTF%2FGyoBzAQp5TAIhkCY81pQI5HrS30Eq4wMI9P%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARACGgw1MDIyOTcwNzYxNjMiDNdS3qDaA5jf2CAaACq3AxenCwSYnYmGBHyh8mTrh4dH7K7B2pK1w5Mr0yBy%2B13GtX3FaWZ11GkGeF%2Fq0dQ%2Fqrcwh%2FsQ2HJzCAno%2BXI2j5ePVLivmvcYwPY0l8TBWi7WTtSSPIQcbj5i4kX77TxKnXwg7ZPNZ8NqIvaq%2BRI9uFLv7rAdYrTLmBXDOpoZ9%2FG%2BCt0Rl5Z5DzZY%2FaQMF5b5VDN5JqxECQG25O09VldrO9tiJWN6Oo%2F%2Fh8qeukp%2BKrMR7c6JhpWOIDX0CKJA9RP9Qg3D6Kom0ozdF7RToPg02uZTciT6gwEBXpiszWFXLrI%2Flqqbqs1VGasVep%2BynxQZIIgv8I3VTVN9FHggaMRXA8hkB4Dx3vHkRA%2B4hGRrlrvFayG0K7y7VyHDLA0YNPa9p7ZYIrgmXwyKdE9jSAAkERNp9Tcqh5J0dhnNo2hSC77TAQFMfmfzKXDIrYUMKTWNX39L9sKnOUGryTdwMuOyK8v1pPZjFzRmiS968ncxClQZZZE1853Tco9z